In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd
import matplotlib.pyplot as plt


project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.kpi import *

/Users/sam11/Git/nestor


In [2]:
def userinput(dict, node):
    
    if node in dict.keys():
        print(node)

        for properties in dict[node]: print("\t" + properties)
        property = input('Which property for the node? ' + node +'\n\t')

        operation = input('Which operation for the property "' + properties + '" ?\n\t')

        value = input('Which value for the property "' + properties + '" ?\n\t')

        return property , operation, value
    else:
        print(node, ' does not exists in your database')
        return None, None, None

# Database

In [3]:
user = "neo4j"
password="GREYSTONE!!"

In [9]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J("bolt://127.0.0.1:7687", user, password, databaseSchema_dict)

yaml file open


In [10]:
dict = database.getAllNode()

------

# Create the objects

## Issue

In [ ]:
prop, oper, val =  userinput(dict, "issue")
issue = OperandIssue(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(issue)

## Human

In [ ]:
prop, oper, val =  userinput(dict, "human")
human = OperandHuman(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(human)

## Technician

In [ ]:
prop, oper, val =  userinput(dict, "technician")
technician = OperandTechnician(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(technician)

## Operator

In [ ]:
prop, oper, val =  userinput(dict, "operator")
operator = OperandOperator(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(operator)

## Machine

In [ ]:
prop, oper, val =  userinput(dict, "machine")
machine = OperandMachine(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(machine)

## Machine Type

In [ ]:
prop, oper, val =  userinput(dict, "type of machine")
machinetype = OperandMachine(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(machinetype)

## Tags

In [ ]:
prop, oper, val =  userinput(dict, "all tags")
tag = OperandTag(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(tag)

## Tags - OneGram

In [ ]:
prop, oper, val =  userinput(dict, "tags one_gram")
one_gram = OperandTagOnegram(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(one_gram)

## Tags - NGram

In [ ]:
prop, oper, val =  userinput(dict, "tags n_gram")
n_gram = OperandTagNgram(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(n_gram)

## Tags - Other

In [ ]:
prop, oper, val =  userinput(dict, "tags other")
other = OperandTechnician(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(other)

## Tags - OneGram - Problem

In [ ]:
prop, oper, val =  userinput(dict, "tags problem")
problem = OperandTagProblem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(problem)

## Tags - OneGram - Solution

In [ ]:
prop, oper, val =  userinput(dict, "tags solution")
solution = OperandTagSolution(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(solution)

## Tags - Other - NA

In [ ]:
prop, oper, val =  userinput(dict, "tags na")
na = OperandTagNa(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(na)

## Tags - Other - StopWord

In [ ]:
prop, oper, val =  userinput(dict, "tags stop_word")
stop_word = OperandTagStopWord(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(stop_word)

## Tags - NGram - ProblemItem

In [ ]:
prop, oper, val =  userinput(dict, "tags problem_item")
problem_item = OperandTagProblemItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(problem_item)

## Tags - NGram - SolutionItem

In [ ]:
prop, oper, val =  userinput(dict, "tags solution_item")
solution_item = OperandTagSolutionItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(solution_item)

## Tags - OneGram - Item

In [ ]:
prop, oper, val =  userinput(dict, "tags item")
item = OperandTagItem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item)

## Tags - OneGram - Item as problem

In [ ]:
prop, oper, val =  userinput(dict, "tag item as problem")
item_problem = OperandTagItemAsProblem(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item_problem)

## Tags - OneGram - Item as solution

In [ ]:
prop, oper, val =  userinput(dict, "tag item as solution")
item_solution = OperandTagItemAsSolution(property = prop, operator =oper, value=val, databaseInfo = database.schema)
print(item_solution)

------

# RETURN

In [ ]:
### Tag Problem
res1 = OperandTagProblem(result = "keyword", databaseInfo = database.schema)
res2 = OperandTagItem(result = "keyword", databaseInfo = database.schema)

# KPI

In [ ]:
query = machine + technician + res1 + res2

In [ ]:
resultToObservationDataframe(database.runQuery(query.cypher_filterQuery())[1])

In [ ]:
print(query.cypher_filterQuery())

------

# KPI from "Developing meintenance key performance indocators from maintenance work order data"

In [11]:
itemAsProblem = OperandTagItemAsProblem(result = "keyword", databaseInfo = database.schema)
machine = OperandMachine(result = "name", databaseInfo = database.schema)
machinetype = OperandMachine(result = "type", databaseInfo = database.schema)
technician = OperandTechnician(result = "name", databaseInfo = database.schema)
problemitem = OperandTagProblemItem(result = "keyword", databaseInfo = database.schema)
problem = OperandTagProblem(result = "keyword", databaseInfo = database.schema)

issue_dateMWOstart = OperandIssue(result = "date_maintenanceworkorder_start", databaseInfo = database.schema)
issue_dateMWOend = OperandIssue(result = "date_maintenanceworkorder_completion", databaseInfo = database.schema)

In [ ]:
print(kpi.cypher_filterQuery())

### 2.1.1 Common Problem Items

In [12]:
kpi = itemAsProblem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

1904it [00:00, 203676.57it/s]


,tagitemasproblem.keyword
0,station
1,station
2,station
3,station
4,station


### 2.1.2 Common Problem Items per Time Between Failure

In [13]:
kpi = itemAsProblem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

1904it [00:00, 169578.38it/s]


,issue.date_maintenanceworkorder_start,tagitemasproblem.keyword,issue.date_maintenanceworkorder_completion
0,2016-07-14 05:30:00,station,2016-07-14 08:00:00
1,2016-02-01 00:00:00,station,2016-02-01 00:00:00
2,2015-03-31 00:00:00,station,2015-03-31 00:00:00
3,2016-06-12 00:00:00,station,2016-06-13 12:00:00
4,2016-01-22 00:00:00,station,2016-01-22 00:00:00


### 2.1.3 Common Problem Items by Machine Type

In [14]:
kpi = itemAsProblem + machinetype
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

0it [00:00, ?it/s]


,machinetype.type,tagitemasproblem.keyword


### 2.1.4 Common Problem Items & Action per Time to Repair

In [15]:
kpi = problemitem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

2089it [00:00, 171734.63it/s]


,tagproblemitem.keyword,issue.date_maintenanceworkorder_start,issue.date_maintenanceworkorder_completion
0,hydraulic leak,2015-03-19 00:00:00,2015-04-22 00:00:00
1,hydraulic leak,2016-09-20 05:30:00,2016-09-22 16:00:00
2,hydraulic leak,2015-12-07 00:00:00,2015-12-07 00:00:00
3,hydraulic leak,2017-01-11 06:15:00,2017-01-11 10:00:00
4,hydraulic leak,2017-01-17 06:15:00,2017-01-18 10:30:00


### 2.2.1 Machine per Time Between Failure

In [16]:
kpi = machine + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

3431it [00:00, 183201.45it/s]


,machine.name,issue.date_maintenanceworkorder_start,issue.date_maintenanceworkorder_completion
0,h27,2017-01-24 00:00:00,2017-01-24 00:00:00
1,h27,2017-01-23 00:00:00,2017-01-23 00:00:00
2,h27,2017-01-13 06:00:00,2017-01-13 07:30:00
3,h27,2017-01-05 12:00:00,None
4,h27,2016-12-23 08:30:00,2017-12-23 00:00:00


### 2.2.2 Machine by Problem Action per Time Between Failure

In [17]:
kpi = machine + problem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

2851it [00:00, 158987.95it/s]


,tagproblem.keyword,machine.name,issue.date_maintenanceworkorder_start,issue.date_maintenanceworkorder_completion
0,broken,h5,2016-06-04 00:00:00,2016-06-04 00:00:00
1,broken,h4,2016-04-13 00:00:00,2016-04-13 00:00:00
2,broken,i1,2016-05-05 00:00:00,2016-05-06 00:00:00
3,broken,h22,2016-07-21 05:30:00,2016-07-21 07:00:00
4,broken,h2,2016-05-04 00:00:00,2016-05-04 00:00:00


### 2.2.3 Machine Type per Time to Repair

In [18]:
kpi = machinetype + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

0it [00:00, ?it/s]


,machinetype.type,issue.date_maintenanceworkorder_start,issue.date_maintenanceworkorder_completion


### 2.2.4 Machine type per Problem Item per Time to Repair

In [19]:
kpi = machinetype +  itemAsProblem + issue_dateMWOstart + issue_dateMWOend
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

0it [00:00, ?it/s]


,machinetype.type,issue.date_maintenanceworkorder_start,tagitemasproblem.keyword,issue.date_maintenanceworkorder_completion


### 2.3.1 Maintenance Technician Expertise per Problem Item

In [20]:
kpi = technician + itemAsProblem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

1314it [00:00, 180474.01it/s]


,technician.name,tagitemasproblem.keyword
0,steve andreozzi,station
1,jon cousineau,station
2,marcel lefebvre,station
3,steve andreozzi,station
4,steve andreozzi,station


### 2.3.2 Maintenance Technician Expertise per Problem Action

In [21]:
kpi = technician + problem
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

2018it [00:00, 195408.18it/s]


,tagproblem.keyword,technician.name
0,broken,john murray
1,broken,anthony paolillo
2,broken,jon cousineau
3,broken,john murray
4,broken,anthony paolillo


### 2.3.3 Maintenance Technician Expertise per Problem Item & Action per Time to Repair
### 2.3.4 Maintenance Technician Expertise per Problem Item & Action per Time to Diagnose
### 2.3.5 Maintenance Technician Expertise per Solution Item & Action per Time to Fix


In [22]:
kpi = technician + problemitem + issue_dateMWOstart + issue_dateMWOend 
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

1471it [00:00, 157699.14it/s]


,tagproblemitem.keyword,technician.name,issue.date_maintenanceworkorder_start,issue.date_maintenanceworkorder_completion
0,hydraulic leak,marcel lefebvre,2015-03-19 00:00:00,2015-04-22 00:00:00
1,hydraulic leak,john murray,2015-12-07 00:00:00,2015-12-07 00:00:00
2,hydraulic leak,anthony paolillo,2015-12-07 00:00:00,2015-12-07 00:00:00
3,hydraulic leak,colin doherty,2015-07-15 00:00:00,2015-07-20 00:00:00
4,hydraulic leak,jon cousineau,2016-04-21 00:00:00,2016-04-22 00:00:00


### 2.3.6 Maintenance Technician Expertise per Machine Type per Time to Repair

In [23]:
kpi = technician + machinetype + issue_dateMWOstart + issue_dateMWOend 
df = resultToObservationDataframe(database.runQuery(kpi.cypher_filterQuery())[1])
df.head()

0it [00:00, ?it/s]


,technician.name,issue.date_maintenanceworkorder_start,machinetype.type,issue.date_maintenanceworkorder_completion
